<a href="https://colab.research.google.com/github/jmcaussade/Arreglo-Incongruencias-Juegos-Jardin/blob/main/Incongruencias_Michel_all_bricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter
!pip install matplotlib

In [22]:
import pandas as pd
import numpy as np
import xlsxwriter
import unicodedata
import matplotlib

# Functions
def generate_colors(n):
    """Generate n distinct colors."""
    cmap = matplotlib.colormaps['tab20']
    return [matplotlib.colors.to_hex(cmap(i/n)) for i in range(n)]

def normalizar_texto(texto):
    """Normalize text - convert to lowercase and remove accents."""
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    return texto

# Read Excel File
archivo_excel = 'Analisis.xlsx'
df = pd.read_excel(archivo_excel, sheet_name='Hoja1')

# Define columns to exclude
excluir_columnas = ["<ID>", "SKU - Chile", "Jefe de Línea", 'Descripción Web', '<Name>', "Marca", "Origen", "<Parent ID>"]

# Generate colors for columns
initial_col_count = max(len(df.columns) - len(excluir_columnas), 1)
colores_columnas = generate_colors(initial_col_count)

# Process DataFrame
for i, col in enumerate(df.columns):
    if col not in excluir_columnas:
        insert_position = df.columns.get_loc(col) + 1
        col_aparece = f"{col} aparece"
        df.insert(insert_position, col_aparece, df.apply(lambda row: normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['<Name>'])) or
                                                        normalizar_texto(str(row[col])) in
                                                        normalizar_texto(str(row['Descripción Web'])), axis=1))

# Calculate Row-wise Error Percentage
boolean_columns = [col for col in df.columns if 'aparece' in col]
df['Error percentage (Row)'] = df[boolean_columns].apply(lambda row: (row.value_counts().get(False, 0) * 100) / len(row), axis=1)

# Calculate Column-wise Error Percentage
error_percentages_col = {}
for col in boolean_columns:
    false_count = df[col].value_counts().get(False, 0)
    total_count = len(df[col])
    error_percentages_col[col] = (false_count * 100) / total_count if total_count else 0

# Append Column-wise Error Percentages as a new row
error_row = {col: error_percentages_col.get(col, np.nan) for col in df.columns}
df = pd.concat([df, pd.DataFrame([error_row])], ignore_index=True)


# Export to Excel
archivo_salida = 'test_codigo1.xlsx'
writer = pd.ExcelWriter(archivo_salida, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)

# Formatting
workbook = writer.book
worksheet = writer.sheets['Sheet1']
format_verdadero = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
format_falso = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})

for col_num, value in enumerate(df.columns.values):
    if value not in excluir_columnas and 'aparece' not in value:
        color_index = (col_num // 2) % len(colores_columnas)
        cell_format = workbook.add_format({'bg_color': colores_columnas[color_index]})
        worksheet.set_column(col_num, col_num, None, cell_format)
    elif 'aparece' in value:
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': True,
                                      'format': format_verdadero})
        worksheet.conditional_format(1, col_num, len(df), col_num,
                                     {'type': 'cell',
                                      'criteria': '=',
                                      'value': False,
                                      'format': format_falso})

# Close and save Excel writer
writer.close()
print(f"Se ha exportado el DataFrame con formato a '{archivo_salida}'.")


Se ha exportado el DataFrame con formato a 'test_codigo1.xlsx'.


<ipython-input-22-05021dca8eef>:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(error_row, ignore_index=True)


<h1>DISPLAY FILE </h1>


In [ ]:
import pandas as pd

# Replace 'your_excel_file.xlsx' with the path to your Excel file
# and 'Sheet1' with the name of the sheet you want to display
file_path = 'test_codigo1.xlsx'
sheet_name = 'Sheet1'

df = pd.read_excel(file_path, sheet_name=sheet_name)

# Display the DataFrame
# You can use df.head() to display the first few rows
# or df to display the entire DataFrame (be cautious with large DataFrames)
df.head()
